##### 版權 2024 Google LLC.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 JAX 和 Flax 在 Gemma 上進行推論


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemma/docs/jax_inference"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />在 ai.google.dev 上檢視</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/doggy8088/generative-ai-docs/blob/main/site/zh/gemma/docs/jax_inference.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中執行</a>
  </td>
  <td>
    <a target="_blank" href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/google/generative-ai-docs/main/site/en/gemma/docs/jax_inference.ipynb"><img src="https://ai.google.dev/images/cloud-icon.svg" width="40" />在 Vertex AI 中開啟</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/doggy8088/generative-ai-docs/blob/main/site/zh/gemma/docs/jax_inference.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />在 GitHub 上檢視原始程式碼</a>
  </td>
</table>


## 概觀

Gemma 是基於 Google DeepMind Gemini 研究技術的輕量級、最先進的開放大型語言模型系列。本教學示範如何使用 [Google DeepMind 的 `gemma` 函式庫](https://github.com/google-deepmind/gemma) 對 Gemma 2B Instruct 模型進行基本取樣/推理。函式庫是以 [JAX](https://jax.readthedocs.io)(高性能數值運算函式庫)、[Flax](https://flax.readthedocs.io)(基於 JAX 的神經網路函式庫)、[Orbax](https://orbax.readthedocs.io/)(用於訓練工具的基於 JAX 的函式庫，例如檢查點) 和 [SentencePiece](https://github.com/google/sentencepiece)(分詞器/去分詞器函式庫) 編寫。儘管此筆記本沒有直接使用 Flax，但 Flax 已用於建立 Gemma。

此筆記本可在 Google Colab 上執行，配備免費 T4 GPU (前往**編輯** > **筆記設定** > 在**硬體加速器** 底下選擇**T4 GPU** )。


## 設定


### 1. 為 Gemma 設定 Kaggle 存取權

要完成本教學課程，需要先按照 [Gemma 設定](https://ai.google.dev/gemma/docs/setup) 中的設定說明進行操作，了解如何執行下列動作：

* 在 [kaggle.com](https://www.kaggle.com/models/google/gemma/) 上取得 Gemma 存取權。
* 選取具有足夠資源來執行 Gemma 模型的 Colab 執行時期。
* 產生並設定 Kaggle 使用者名稱和 API 金鑰。

完成 Gemma 設定後，請移至下一章節，設定 Colab 環境的環境變數。

### 2. 設定環境變數

設定 `KAGGLE_USERNAME` 和 `KAGGLE_KEY` 的環境變數。當出現「授予存取權？」訊息時，請同意提供機密存取權。


In [1]:
import os
from google.colab import userdata # `userdata` is a Colab API.

os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

### 3. 安裝 `gemma` 函式庫

本筆記本專注於使用免費的 Colab GPU。若要啟用硬體加速，請按一下 **編輯** > **筆記本設定** > 選取 **T4 GPU** > **儲存** 。

接著，你需要從 [`github.com/google-deepmind/gemma`](https://github.com/google-deepmind/gemma) 安裝 Google DeepMind 的 `gemma` 函式庫。如果你收到關於「pip 的相依關系解析器」錯誤，通常可以忽略它。

**注意：** 安裝 `gemma` 後，你也會安裝 [`flax`](https://flax.readthedocs.io)、[`jax`](https://jax.readthedocs.io) 核心、[`optax`](https://optax.readthedocs.io/en/latest/)(JAX-based 的梯度處理和最佳化函式庫)、[`orbax`](https://orbax.readthedocs.io/)、[`sentencepiece`](https://github.com/google/sentencepiece)。


In [2]:
!pip install -q git+https://github.com/google-deepmind/gemma.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.4/244.4 kB 13.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires absl-py<2.0.0,>=0.9, but you have absl-py 2.1.0 which is incompatible.


## 載入並準備 Gemma 模型

1. 使用 [`kagglehub.model_download`](https://github.com/Kaggle/kagglehub/blob/bddefc718182282882b72f814d407d89e5d178c4/src/kagglehub/models.py#L12) 載入 Gemma 模型，此函式需要三個參數：

- `handle`：Kaggle 的模型代號
- `path`：(Optional string) 本機路徑
- `force_download`：(Optional boolean) 強制重新載入模型

**注意：** Gemma 模型大約有 3.7Gb 大。


In [3]:
GEMMA_VARIANT = '2b-it' # @param ['2b', '2b-it'] {type:"string"}

In [4]:
import kagglehub

GEMMA_PATH = kagglehub.model_download(f'google/gemma/flax/{GEMMA_VARIANT}')

100%|██████████| 3.67G/3.67G [00:35<00:00, 110MB/s]
Extracting model files...


In [5]:
print('GEMMA_PATH:', GEMMA_PATH)

GEMMA_PATH: /root/.cache/kagglehub/models/google/gemma/flax/2b-it/2


**提示：** 上方的輸出路徑是模型權重和 tokenizer 本機儲存的位置，你稍後會用到它們。


2. 檢查模型權重和 tokenizer 的位置，然後設定路徑變數。tokenizer 目錄將位於你下載模型的主目錄中，而模型權重則位於子目錄中。例如：

- `tokenizer.model` 檔案將位於 `/LOCAL/PATH/TO/gemma/flax/2b-it/2`)。
- 模型檢查點將位於 `/LOCAL/PATH/TO/gemma/flax/2b-it/2/2b-it`)。


In [6]:
CKPT_PATH = os.path.join(GEMMA_PATH, GEMMA_VARIANT)
TOKENIZER_PATH = os.path.join(GEMMA_PATH, 'tokenizer.model')
print('CKPT_PATH:', CKPT_PATH)
print('TOKENIZER_PATH:', TOKENIZER_PATH)

CKPT_PATH: /root/.cache/kagglehub/models/google/gemma/flax/2b-it/2/2b-it
TOKENIZER_PATH: /root/.cache/kagglehub/models/google/gemma/flax/2b-it/2/tokenizer.model


## 進行取樣/推論


1. 使用[`gemma.params.load_and_format_params`](https://github.com/google-deepmind/gemma/blob/c6bd156c246530e1620a7c62de98542a377e3934/gemma/params.py#L27) 方法載入並格式化 Gemma 模型檢查點:


In [7]:
from gemma import params as params_lib

params = params_lib.load_and_format_params(CKPT_PATH)

2. 載入 Gemma tokenizer，結構化使用 [`sentencepiece.SentencePieceProcessor`](https://github.com/google/sentencepiece/blob/4d6a1f41069c4636c51a5590f7578a0dbed83450/python/src/sentencepiece/__init__.py#L423)：


In [8]:
import sentencepiece as spm

vocab = spm.SentencePieceProcessor()
vocab.Load(TOKENIZER_PATH)

True

3. 若要從 Gemma 模型檢查點自動載入正確設定，請使用 [`gemma.transformer.TransformerConfig`](https://github.com/google-deepmind/gemma/blob/56e501ce147af4ea5c23cc0ddf5a9c4a6b7bd0d0/gemma/transformer.py#L65)。 `cache_size` 參數是在 Gemma `Transformer` 快取中的時間步驟數目。之後，使用 [`gemma.transformer.Transformer`](https://github.com/google-deepmind/gemma/blob/56e501ce147af4ea5c23cc0ddf5a9c4a6b7bd0d0/gemma/transformer.py#L136) 將 Gemma 模型例項化為 `transformer` (繼承自 [`flax.linen.Module`](https://flax.readthedocs.io/en/latest/api_reference/flax.linen/module.html))。

**注意：** 因為在目前的 Gemma 發行版本中沒有使用 Token，所以詞彙量小於輸入嵌入的數量。


In [9]:
from gemma import transformer as transformer_lib

transformer_config = transformer_lib.TransformerConfig.from_params(
    params=params,
    cache_size=1024
)

transformer = transformer_lib.Transformer(transformer_config)

3. 在 Gemma 模型檢查點/權重和分詞器之上使用 [`gemma.sampler.Sampler`](https://github.com/google-deepmind/gemma/blob/c6bd156c246530e1620a7c62de98542a377e3934/gemma/sampler.py#L88) 建立一個 `sampler`：


In [10]:
from gemma import sampler as sampler_lib

sampler = sampler_lib.Sampler(
    transformer=transformer,
    vocab=vocab,
    params=params['transformer'],
)

4. 在 `input_batch` 中撰寫提示並執行推斷。你可以調整 `total_generation_steps` (生成回應時執行的步驟數 — 此範例使用 `100` 來保留主機記憶體)。

**注意：** 如果你用完記憶體，請按一下 **Runtime** > **斷開連接並刪除執行時期** ，然後按一下 **Runtime** > **全部執行** 。


In [11]:
prompt = [
    "\n# What is the meaning of life?",
]

reply = sampler(input_strings=prompt,
                total_generation_steps=100,
                )

for input_string, out_string in zip(prompt, reply.text):
    print(f"Prompt:\n{input_string}\nOutput:\n{out_string}")

Prompt:

# What is the meaning of life?
Output:


The question of what the meaning of life is one that has occupied the minds of philosophers, theologians, and individuals for centuries. There is no single, universally accepted answer, but there are many different perspectives on this complex and multifaceted question.

**Some common perspectives on the meaning of life include:**

* **Biological perspective:** From a biological standpoint, the meaning of life is to survive and reproduce.
* **Existential perspective:** Existentialists believe that life is not inherently meaningful and that


5. (選填) 如果你已完成筆記本並想嘗試另一個提示，請執行此單元格來釋放記憶體。之後，你可以在步驟 3 中再次實例化「抽樣器」，並在步驟 4 中自訂並執行提示。


In [ ]:
del sampler

## 瞭解更多

- 你可以在 [GitHub 上的 Google DeepMind [`gemma` 函式庫](https://github.com/google-deepmind/gemma) 中進一步瞭解更多](https://github.com/google-deepmind/gemma/blob/main/gemma/params.py)，其中包含本教學課程中使用的模組文件字串，例如 [`gemma.params`](https://github.com/google-deepmind/gemma/blob/main/gemma/params.py)、
[`gemma.transformer`](https://github.com/google-deepmind/gemma/blob/main/gemma/transformer.py) 和
[`gemma.sampler`](https://github.com/google-deepmind/gemma/blob/main/gemma/sampler.py)。
- 以下函式庫有其各自的說明文件網站： [核心 JAX](https://jax.readthedocs.io)、[Flax](https://flax.readthedocs.io) 和 [Orbax](https://orbax.readthedocs.io/)。
- 對於 `sentencepiece` 分詞器/還原文件說明，請查看 [Google 的 `sentencepiece` GitHub 存放庫](https://github.com/google/sentencepiece)。
- 對於 `kagglehub` 說明文件，請查看 [Kaggle 的 `kagglehub` GitHub 存放庫](https://github.com/Kaggle/kagglehub) 上的 `README.md`。
- 瞭解如何 [將 Gemma 模型與 Google Cloud Vertex AI 整合使用](https://cloud.google.com/vertex-ai/docs/generative-ai/open-models/use-gemma)。
